In [ ]:
from pynq import Overlay
import pynq.lib.dma

In [ ]:
ol = Overlay("design_1.bit")

In [ ]:
#until including this cell: run always, even just with CFFI
mkl_gpio0 = ol.axi_gpio_0
mkl_gpio1 = ol.axi_gpio_1
mkl_div = ol.clk_wiz_0

In [ ]:
XCLK_CLK_CFG_2 = 0x208
XCLK_CLK_CFG_23 = 0x25C

XGPIO_DATA0 = 0x0
XGPIO_DATA2 = 0x8

CLK_DIV = 0x100       #0x000, 0x100, 0x200, ...

#it does not speed up, slower!
VAL_32BIT_WR  = 0x8000000E | CLK_DIV
VAL_32BIT_WRn = 0x0000000E | CLK_DIV
VAL_24BIT_WR  = 0x8000001E | CLK_DIV
VAL_24BIT_WRn = 0x0000001E | CLK_DIV
VAL_TA_WR     = 0x8000002E | CLK_DIV
VAL_TA_WRn    = 0x0000002E | CLK_DIV
VAL_32BIT_RD  = 0x8000004E | CLK_DIV
VAL_32BIT_RDn = 0x0000004E | CLK_DIV

TRG_CNT = 0

def trigger_cnt(inc):
    global TRG_CNT;
    if inc :
        TRG_CNT = TRG_CNT + 1
    return (TRG_CNT & 0x3) << 30
    
def cnfg0(QSPIio0):
    return 0

def cnfg1(QSPIio1):
    return 0

def cnfgdiv(ClkDiv):
    ClkDiv.write(XCLK_CLK_CFG_2, 0x10)                  #0x10 = 25 MHz, 0x40 = 6.25 MHz
    ClkDiv.write(XCLK_CLK_CFG_23, 0x03)
    return 0

def QSPI_Write32bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    mkl_gpio1.write(XGPIO_DATA0, 0x0000010E | trigger_cnt(1))

def QSPI_Write32bitLE(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    mkl_gpio1.write(XGPIO_DATA0, 0x0000018E | trigger_cnt(1))
        
def QSPI_Write24bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    mkl_gpio1.write(XGPIO_DATA0, 0x0000011E | trigger_cnt(1))
        
def QSPI_TurnAround():
    mkl_gpio1.write(XGPIO_DATA0, 0x0000012E | trigger_cnt(1))
    
def QSPI_Read32bit():
    mkl_gpio1.write(XGPIO_DATA0, 0x0000014E | trigger_cnt(1))
    s = mkl_gpio0.read(XGPIO_DATA2)
    return s

def QSPI_Read32bitLE():
    mkl_gpio1.write(XGPIO_DATA0, 0x000001CE | trigger_cnt(1))
    s = mkl_gpio0.read(XGPIO_DATA2)
    return s
    
def QSPI_Deselect():
    global TRG_CNT;
    mkl_gpio1.write(XGPIO_DATA0, 0x0040010F | trigger_cnt(0))  #release all nCS signals
    TRG_CNT = 0

In [ ]:
cnfg0(mkl_gpio0)
cnfg1(mkl_gpio1)
cnfgdiv(mkl_div)

In [ ]:
QSPI_Deselect()
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345670)
QSPI_Write24bit(0xABCDEF00)        #is val << 8
QSPI_Write32bitLE(0x98765400)      #N-times
QSPI_Write32bitLE(0xACBD2013)
QSPI_Write32bitLE(0xBCDEF012)
QSPI_Deselect()

In [ ]:
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345678)
QSPI_Write24bit(0xABCDFE00)        #is val << 8
QSPI_TurnAround()
v1 = QSPI_Read32bitLE()            #N-times
v2 = QSPI_Read32bitLE()
v3 = QSPI_Read32bitLE()
QSPI_Deselect()
print(hex(v1))
print(hex(v2))
print(hex(v3))

In [ ]:
from cffi import FFI
import array

wrVal = array.array('I', [0xE0, 0x11223344, 0x556677, 0x98765432, 0x12345678, 0x11223344, 0x55667788])

ffi = FFI()
ffi.cdef("""unsigned int C_QSPI_Init(int, int, unsigned int);
            unsigned int C_QSPI_Write(unsigned int *, int);
            void C_QSPI_Deinit(void);
         """)
clib = ffi.dlopen("/home/xilinx/c_code/libGeneric_QSPI.so")
i = clib.C_QSPI_Init(2, 0, 0x3F)      #2 is max! = 16.6 MHz if without flow control
#print(i)
i = clib.C_QSPI_Write(ffi.cast("unsigned int *", ffi.from_buffer(wrVal)), len(wrVal))

clib.C_QSPI_Deinit()
print(i)

In [ ]:
from cffi import FFI
import array

wrVal = array.array('I', [0xE1, 0x11223344, 0x556677])
rdVal = array.array('I', [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

ffi = FFI()
ffi.cdef("""unsigned int C_QSPI_Init(int, int, unsigned int);
            unsigned int C_QSPI_Read(unsigned int *, unsigned int *, int);
            void C_QSPI_Deinit(void);
        """)
clib = ffi.dlopen("/home/xilinx/c_code/libGeneric_QSPI.so")
i = clib.C_QSPI_Init(3, 0, 0x7F)
i = clib.C_QSPI_Read(ffi.cast("unsigned int *", ffi.from_buffer(wrVal)), ffi.cast("unsigned int *", ffi.from_buffer(rdVal)), len(rdVal))
clib.C_QSPI_Deinit()
print(i)
print(rdVal)


In [ ]:
#change the PLL clock for AXI bus and PL - decreases the Rx sample delay!
core = pynq.ps.Clocks
print(core.cpu_mhz)
print(core.fclk0_mhz)
core.fclk0_mhz = 333.3
print(core.fclk0_mhz)

In [ ]:
#read INT inputs:
s = mkl_gpio1.read(XGPIO_DATA2)
print(hex(s))

In [ ]:
#GPIO via C-code library:
from cffi import FFI

ffi = FFI()
ffi.cdef("""unsigned int C_QSPI_Init(int, int, unsigned int);
            void C_GPIO_Out(unsigned int);
            unsigned int C_GPIO_In(void);
            void C_QSPI_Deinit(void);
        """)
clib = ffi.dlopen("/home/xilinx/c_code/libGeneric_QSPI.so")
i = clib.C_QSPI_Init(3, 0, 0x7F)
clib.C_GPIO_Out(0x35)
i = clib.C_GPIO_In()
clib.C_QSPI_Deinit()
print(i)

In [ ]:
#I2C test: I2C Extender, all as input
from pynq.lib.iic import AxiIIC
iic = AxiIIC(ol.ip_dict["axi_iic_0"])
wr_data = [0]
rd_data = bytes(2)   #it must be bytes()!!
iic.send(0x76, wr_data, len(wr_data), 0)
iic.receive(0x76, rd_data, 2, 0)
print(rd_data)

In [ ]:
#I2C test: I2C Extender, port as output
wr_data = [6, 0x00]
iic.send(0x76, wr_data, len(wr_data), 0)
wr_data = [0x02, 0xFF]
iic.send(0x76, wr_data, len(wr_data), 0)

In [ ]:
#test Generic_QSPI.py as LIB:
import Generic_QSPI as QSPI
import array

i = QSPI.QSPIInit()
print(i)

tr = array.array('I', [0xE0, 0x11223344, 0x556677, 0x98765432, 0x12345678, 0x11223344, 0x55667788])
QSPI.SendTransaction(tr, 0)

r = QSPI.GPIOin()
print(r)
QSPI.GPIOout(0x35)

tr = array.array('I', [0xE1, 0x11223344, 0x556677])
r = QSPI.SendTransaction(tr, 4)
print(r)

In [ ]:
#test HCC QSPI Python as LIB:
import HCC_QSPI as HCC

In [ ]:
HCC.Main()